In [1]:
# Import statements
import os
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd
import time
from langchain_community.document_loaders import PyMuPDFLoader


In [ ]:
from datetime import datetime


In [ ]:
"""
# add func for adding the 
1. staus                :done
2. pdf_count            :done
5. index of the file    :done
6. is pdf readable using langchain pdf reader: check using length
==============================================================================================================================
TO-DO:
- re-run the code: compare with original file name rather than new file name: if same; version them and save
-  remove duplicates if any
- parse pdf and get data: 
=== choose 1 review as sample size : "Clinical Pharmacology Biopharmaceutics Review(s)",
- segment data:
- get review: comments or sections or any possible entry and store

"""

In [9]:
base_path="/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/new_updated_files"
# folder_name = "pdf_files"
drug_name = "" # insert drug name
json_specific_id = "" # insert json specific id
file = "" # insert file name from web
file_name = drug_name + json_specific_id
# temp_file_path = os.path.join(base_path,folder_name)


# os.path.join()

In [ ]:
# extract json data
extracted_data=[]
for i in data['results']:
    if 'submissions' in i.keys():
        for j in i['submissions']:
            if 'application_docs' in j.keys():
                for k in j['application_docs']:
                    if k['type'] == 'Review':
                        # print(k)
                        extracted_data.append([k['id'], k['url'], k['url'].split("/")[-2]])

url_data = pd.DataFrame(extracted_data, columns = ['json_id', 'url', 'year'])
url_data.to_csv('/Users/shubhamagrawal/Documents/MS fall 25/MS fall\'25/ra/data/url_data.csv', index=False, header=True, sep=',')

In [18]:
# if same name: keep as diff version

def func_download_pdf(filename, pdf_url, headers, save_path):
    try:
        print(f"Downloading: {filename}")
        pdf_response = requests.get(pdf_url, stream=True,headers=headers)
        with open(save_path, 'wb') as f:
            for chunk in pdf_response.iter_content(chunk_size=8192):
                if chunk:
                    f.write(chunk)
        print(f"Saved to: {save_path}")
    except Exception as e:
        print(f"Failed to download {pdf_url}: {e}")


def download_pdfs(url, keywords, download_path, json_id, year, keyword_map):
    # Create folder if it doesn't exist
    os.makedirs(download_path, exist_ok=True)
    
    headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
        }

    
    # Get webpage content
    status = "Success"
    pdf_count = 0
    path_index = ''
    drug_type = ''
    length = 0
    original_filename = ""
    filename=''
    filepath = ''
    review_type=''
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to access {url}, status code: {response.status_code}")
        status="Failure"
        return status, pdf_count, path_index, drug_type, length, original_filename,review_type, filepath
    else:
        print("success to access url", url)
        
    
    if url.endswith(".pdf"):
        filename = os.path.basename(url)
        original_filename=filename
        folder_name = ('_').join([str(year), str(json_id), "others"])
        save_path = os.path.join(download_path, folder_name, original_filename)
        filepath=save_path
        path_list = save_path.split('/')[-2:]
        path_index = ('_').join(path_list).replace(' ', '_')[:-4]
        drug_type = path_list[0].strip().split('_')[-1].replace(' ','_')
        review_type='No Category'
        os.makedirs(os.path.join(download_path, folder_name), exist_ok=True)
        if os.path.exists(save_path):
            print("exists")
            pass
        else:
            func_download_pdf(original_filename, url, headers, save_path)
        pdf_count +=1
        status = "Success"
        loader = PyMuPDFLoader(save_path)
        # length = 0
        for idx, doc in enumerate(loader.load()):
            length += len(doc.page_content)
        return status, pdf_count, path_index, drug_type, length, original_filename,review_type, filepath
    else:        
        soup = BeautifulSoup(response.text, 'html.parser')
        # print(soup)
        title = soup.find('title')
        if title != None:
            # print(title.string)
            if ":" in title.string.strip():
                drug_name = title.string.strip().split(":")[1].strip().split(" ")[0]
            else:
                drug_name = title.string.strip()
            # print(drug_name)
            folder_name = ('_').join([str(year), str(json_id), drug_name])
            # print(folder_name)        
            links = soup.find_all('a', href=True)
            # print(links)
            
            # Normalizing  keywords
            keywords = [k.lower() for k in keywords]
        
            for link in links:
                href = link['href']
                text = link.get_text(strip=True).lower()
                parent_text = link.find_parent().get_text(strip=True).lower() if link.find_parent() else ""
                parent_text = " ".join(parent_text.split())
                # print(parent_text)
                
        
                # Check if it's a PDF link and matches any keyword
                if href.endswith(".pdf") and any(k in text or k in parent_text for k in keywords):
                    pdf_url = urljoin(url, href)
                    filename = os.path.basename(pdf_url)
                    # original_filename=filename.split(".")[0]
                    matched_name = [k for k in keywords if k in text or k in parent_text]
                    review_type=matched_name[0]
                    
                 
                    # filename = filename.replace(filename.split(".")[0],"_".join(matched_name[0].split(" ")))
                    original_filename = keyword_map.get(matched_name[0],"na") + "_" + filename
                    # "_".join(,original_filename)
                    # print(filename)
                 
                    save_path = os.path.join(download_path, folder_name, original_filename)
                    filepath=save_path
                    # print(save_path)
                    path_list = save_path.split('/')[-2:]
                    path_index = ('_').join(path_list).replace(' ', '_')[:-4]
                    drug_type = path_list[0].strip().split('_')[-1].replace(' ','_')
                    os.makedirs(os.path.join(download_path, folder_name), exist_ok=True)
                    
                   # Download the PDF def func_download_pdf(filename, pdf_url, headers, save_path):
                    if os.path.exists(save_path):
                        print("exists")
                        pass
                    else:
                        func_download_pdf(original_filename, pdf_url, headers, save_path)
                    pdf_count +=1
                    status = "Success"

                    loader = PyMuPDFLoader(save_path)
                    for idx, doc in enumerate(loader.load()):
                        length += len(doc.page_content)
                    # print(idx,length)
                    
        return status, pdf_count, path_index, drug_type, length, original_filename,review_type, filepath





def get_all_subpaths(directory_path):
    """
    Retrieves all subpaths (directories and files) within a given directory
    and its subdirectories.

    Args:
        directory_path (str): The path to the starting directory.

    Returns:
        list: A list of strings, where each string is the full path
              to a subdirectory or file within the given directory tree.
    """
    all_file_paths = []
    for root, _, files in os.walk(directory_path):
        for file in files:
            full_path = os.path.join(root, file)
            all_file_paths.append(full_path)
    return all_file_paths
    




In [23]:
new_data = []
# new_data

In [13]:
url_data = pd.read_csv('/Users/shubhamagrawal/Documents/MS fall 25/MS fall\'25/ra/data/url_data.csv')

url_data.shape

(7660, 3)

In [ ]:


# cols = ['json_id', 'year', 'url',  'url_parsed_status', 'pdf_count', 'path_index', 'drug_type','pdf_content_length','pymupdf_parsaed','original_filename',"review_type", "filepath"]


In [66]:
url_data.head()

,json_id,url,year
0,22545,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002
1,43064,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002
2,43063,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002
3,22547,http://www.accessdata.fda.gov/drugsatfda_docs/...,2004
4,22546,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002


In [10]:
batch = [500,1000,1500,2000,2500,3000,3500,4000,4300,4700,5000,5300,5700,6000,6300, 6700,7000, 7300,7660]
# main_df['original_filename']=''
# main_df.head()

In [24]:
search_keywords = ["Pharmacology Review(s)", 
                   "Statistical Review(s)",
                   "Clinical Pharmacology Biopharmaceutics Review(s)",
                   "Risk Assessment and Risk Mitigation Review(s)",
                   "Immunogenicity Review",
                    "Risk Evaluation and Mitigation Strategy(REMS)"]

keyword_map = {
            "pharmacology review(s)"                          :"pharm_rv", 
            "statistical review(s)"                           :"stat_rv",
            "clinical pharmacology biopharmaceutics review(s)":"cl_pharm_rv",
            "risk assessment and risk mitigation review(s)"   :"ra_rm_rv",
            "immunogenicity review"                           :"imm_rv",
            "risk evaluation and mitigation strategy(rems)"   :"rems_rv"     
}
save_folder = base_path



cols = ['json_id', 'year', 'url',  'url_parsed_status', 'pdf_count', 'path_index', 'drug_type','pdf_content_length',
        'pymupdf_parsaed','original_filename',"review_type", "filepath"]

main_df_1 = pd.DataFrame(columns=cols)
# main_df = pd.read_csv('/Users/shubhamagrawal/Documents/MS fall 25/MS fall\'25/ra/data/main_df.csv')


In [21]:
[k.lower() for k in search_keywords]

['pharmacology review(s)',
 'statistical review(s)',
 'clinical pharmacology biopharmaceutics review(s)',
 'risk assessment and risk mitigation review(s)',
 'immunogenicity review',
 'risk evaluation and mitigation strategy(rems)']

In [ ]:
## do not use unless new URLS

for idx,row in url_data[3500:].iterrows(): 
    print("============================  checking url:", idx, "================================ ")
    if idx in [4123,4687,5787]:
        continue
    else:
        url = row.url
        json_id = row.json_id
        year = row.year
        webpage_url = url
        # print(url,json_id)
        stat,pdf_cnt, path_index, drug_type, length, original_filename,review_type, filepath = download_pdfs(
                                                                                    webpage_url, 
                                                                                    search_keywords,
                                                                                    save_folder,
                                                                                    json_id,
                                                                                    year,
                                                                                    keyword_map
                                                                                   )
        parsed = 0
        if length > 0:
            parsed = 1
        else:
            parsed = 0
        # print(idx, stat, pdf_cnt,drug_type)
        # print("checkpoint:",original_filename, filepath)
        # print(stat, pdf_cnt)
        new_data.append([json_id, year, url, stat, pdf_cnt, path_index, drug_type, length, parsed, original_filename,review_type, filepath])
        # print(new_data)
        print("===========================")
        if idx in batch:
            new_df = pd.DataFrame(data = new_data, columns = cols)
            main_df_1 = pd.concat([main_df_1, new_df], ignore_index=True)
            # print(main_df)
            main_df_1.to_csv('/Users/shubhamagrawal/Documents/MS fall 25/MS fall\'25/ra/data/processed_data.csv', index=False, header=True, sep=',')
            print("===================================checkpoint at ", idx, "===================================")


            

============================  checking url: 3500 ================================ 
success to access url http://www.accessdata.fda.gov/drugsatfda_docs/nda/2003/021042_s023_VioxxtmTOC.cfm
===================================checkpoint at  3500 ===================================
============================  checking url: 3501 ================================ 
success to access url http://www.accessdata.fda.gov/drugsatfda_docs/nda/2002/21-042s007_Vioxx.cfm
exists
exists
============================  checking url: 3502 ================================ 
success to access url http://www.accessdata.fda.gov/drugsatfda_docs/nda/2002/021042_S015_VIOXX.pdf
exists
============================  checking url: 3503 ================================ 
success to access url http://www.accessdata.fda.gov/drugsatfda_docs/nda/2003/021042_s020s021s022_021052_s014s015s016_VioxxTOC.cfm
============================  checking url: 3504 ================================ 
success to access url http://www.accessdat

In [71]:
for idx,row in main_df[:21].iterrows(): 
    print("============================  checking url:", idx, "================================ ")
    if idx in [4123,4687,5787]:
        continue
    else:
        url = row.url
        json_id = row.json_id
        year = row.year
        webpage_url = url
        # print(url,json_id)
        stat,pdf_cnt, path_index, drug_type, length, original_filename, filename = download_pdfs(webpage_url, search_keywords, save_folder, json_id, year)
        parsed = 0
        if length > 0:
            parsed = 1
        else:
            parsed = 0
        # print(idx, stat, pdf_cnt,drug_type)
        print("checkpoint:",original_filename)
        main_df.at[idx,'original_filename'] = original_filename
        main_df.at[idx,'modified_filename'] = filename
    
        # print(stat, pdf_cnt)
        # new_data.append([json_id, year, url, stat, pdf_cnt, path_index, drug_type, length, parsed, original_filename])
        # print(new_data)
        # print("===========================")
        # if idx in batch:
        #     new_df = pd.DataFrame(data = new_data, columns = cols)
        #     main_df = pd.concat([main_df, new_df], ignore_index=True)
        #     print(main_df)
        #     main_df.to_csv('/Users/shubhamagrawal/Documents/MS fall 25/MS fall\'25/ra/data/main_df.csv', index=False, header=True, sep=',')
        #     print("===================================checkpoint at ", idx, "===================================")


            

============================  checking url: 0 ================================ 
success to access url http://www.accessdata.fda.gov/drugsatfda_docs/nda/96/alteplase_toc.cfm
checkpoint: 
============================  checking url: 1 ================================ 
success to access url http://www.accessdata.fda.gov/drugsatfda_docs/nda/96/103628Orig1s000rev.pdf
exists
checkpoint: 103628Orig1s000rev.pdf
============================  checking url: 2 ================================ 
success to access url http://www.accessdata.fda.gov/drugsatfda_docs/nda/96/018998_S050_VASOTEC_AP.pdf
exists
checkpoint: 018998_S050_VASOTEC_AP.pdf
============================  checking url: 3 ================================ 
success to access url http://www.accessdata.fda.gov/drugsatfda_docs/nda/96/018998_S034_VASOTEC_AP.pdf
exists
checkpoint: 018998_S034_VASOTEC_AP.pdf
============================  checking url: 4 ================================ 
success to access url http://www.accessdata.fda.gov/drugsa

In [72]:
main_df.head(30)

,json_id,year,url,status,pdf_count,path_index,drug_type,pdf_content_length,pymupdf_parsaed,original_filename,modified_filename
0,48910,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,0,NaN,NaN,0,0,,
1,57420,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_57420_others_103628Orig1s000rev,others,74010,1,103628Orig1s000rev.pdf,103628Orig1s000rev.pdf
2,20189,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_20189_others_018998_S050_VASOTEC_AP,others,0,0,018998_S050_VASOTEC_AP.pdf,018998_S050_VASOTEC_AP.pdf
3,40848,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_40848_others_018998_S034_VASOTEC_AP,others,0,0,018998_S034_VASOTEC_AP.pdf,018998_S034_VASOTEC_AP.pdf
4,48910,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,0,NaN,NaN,0,0,,
5,57420,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_57420_others_103628Orig1s000rev,others,74010,1,103628Orig1s000rev.pdf,103628Orig1s000rev.pdf
6,20189,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_20189_others_018998_S050_VASOTEC_AP,others,0,0,018998_S050_VASOTEC_AP.pdf,018998_S050_VASOTEC_AP.pdf
7,40848,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_40848_others_018998_S034_VASOTEC_AP,others,0,0,018998_S034_VASOTEC_AP.pdf,018998_S034_VASOTEC_AP.pdf
8,41030,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Failure,0,NaN,NaN,0,0,,
9,41029,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Failure,0,NaN,NaN,0,0,,


In [ ]:

cols = ['json_id', 'year', 'url',  'status', 'pdf_count', 'path_index', 'drug_type','pdf_content_length','pymupdf_parsaed']
new_df = pd.DataFrame(data = new_data, columns = cols)

In [14]:
new_df

,json_id,year,url,status,pdf_count,path_index,drug_type,pdf_content_length,pymupdf_parsaed
0,20940,2001,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,2001_20940_others_020779_S036_VIRACEPT,others,0,0
1,41692,2004,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,2004_41692_others_020779_S042_VIRACEPT,others,0,0


In [16]:

cols = ['json_id', 'year', 'url',  'status', 'pdf_count', 'path_index', 'drug_type','pdf_content_length','pymupdf_parsaed']
main_df = pd.DataFrame(columns=cols)
pd.concat([main_df, new_df], 
                    ignore_index=True
                    )

,json_id,year,url,status,pdf_count,path_index,drug_type,pdf_content_length,pymupdf_parsaed
0,20940,2001,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,2001_20940_others_020779_S036_VIRACEPT,others,0,0
1,41692,2004,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,2004_41692_others_020779_S042_VIRACEPT,others,0,0


In [17]:
pd.concat([main_df, new_df], 
                    ignore_index=True
                    )

,json_id,year,url,status,pdf_count,path_index,drug_type,pdf_content_length,pymupdf_parsaed
0,20940,2001,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,2001_20940_others_020779_S036_VIRACEPT,others,0,0
1,41692,2004,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,2004_41692_others_020779_S042_VIRACEPT,others,0,0


In [72]:
file_path = "/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files/96_20686_Topamax/pharmacology_review(s).pdf"


In [73]:
docs = loader.load()
docs[0]

Document(metadata={'producer': 'PDFlib 4.0.1 (Win32)', 'creator': 'ARTS Import', 'creationdate': '2004-09-08T13:01:06-04:00', 'source': "/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files/96_20686_Topamax/pharmacology_review(s).pdf", 'file_path': "/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files/96_20686_Topamax/pharmacology_review(s).pdf", 'total_pages': 77, 'format': 'PDF 1.4', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2004-09-08T13:01:25-04:00', 'trapped': '', 'modDate': "D:20040908130125-04'00'", 'creationDate': "D:20040908130106-04'00'", 'page': 0}, page_content='')

In [84]:
loader = PyMuPDFLoader(file_path)
length = 0
for idx, doc in enumerate(loader.load()):
    length += len(doc.page_content)
print(idx,length)

76 0


In [96]:
url_data.shape

(7660, 3)

In [97]:
7660/2

3830.0

In [66]:
# # Example usage:
# target_directory = "/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files"  # Replace with your actual directory
# subpaths = get_all_subpaths(target_directory)

# for path in subpaths[:20]:
path = "/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files/2005_22550_others/076250_ORIGINAL APPROVAL_PACKAGE.pdf"
temp = path.split('/')[-2:]

# print(('_').join(path.split('/')[-2:]).replace(' ', '_')[:-4])
print(temp)
print(temp[0].strip().split('_')[-1].replace(' ','_'))

['2005_22550_others', '076250_ORIGINAL APPROVAL_PACKAGE.pdf']
others


In [15]:
import pandas as pd
import random
import time

# Create an empty DataFrame with defined columns
main_df = pd.DataFrame(columns=['Iteration', 'Value', 'Timestamp'])

# Temporary batch holder
batch_data = []

# Run 10,000 iterations
for i in range(1, 10001):
    # Simulate some data
    value = random.random()  # random float between 0 and 1
    timestamp = time.strftime('%Y-%m-%d %H:%M:%S')

    # Collect data in a batch list (not DataFrame yet)
    batch_data.append({'Iteration': i, 'Value': value, 'Timestamp': timestamp})

    # Every 200 iterations, append batch to main DataFrame and clear the batch
    if i % 200 == 0:
        batch_df = pd.DataFrame(batch_data)
        main_df = pd.concat([main_df, batch_df], ignore_index=True)
        batch_data = []  # reset batch

        # Optionally, save to CSV file every 200 iterations
        main_df.to_csv('data_progress.csv', index=False)
        print(f"Saved progress at iteration {i}")

# Handle any remaining data if total iterations not divisible by 200
if batch_data:
    batch_df = pd.DataFrame(batch_data)
    main_df = pd.concat([main_df, batch_df], ignore_index=True)

# Final save
main_df.to_csv('data_final.csv', index=False)
print("✅ All iterations complete and data saved to 'data_final.csv'")


In [16]:
df.head()

,json_id,url,year
0,22545,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002
1,43064,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002
2,43063,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002
3,22547,http://www.accessdata.fda.gov/drugsatfda_docs/...,2004
4,22546,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002


In [ ]:
"""
 fetch these reviews
 Pharmacology Review(s) (PDF)
Statistical Review(s) (PDF)
Clinical Pharmacology Biopharmaceutics Review(s) (PDF)
Risk Assessment and Risk Mitigation Review(s) (PDF)
"""

In [29]:
temp_url.replace(temp_url.split("/")[-1],"125276s000StatR.pdf")

'https://www.accessdata.fda.gov/drugsatfda_docs/nda/2010/125276s000StatR.pdf'

In [196]:
print("rgfreg")

rgfreg


============================  checking url: 0 ================================ 


KeyboardInterrupt: 

In [178]:
url_data[120:]

,json_id,url,year,status
120,20175,http://www.accessdata.fda.gov/drugsatfda_docs/...,pre96,Success
121,40844,http://www.accessdata.fda.gov/drugsatfda_docs/...,pre96,Success
122,40842,http://www.accessdata.fda.gov/drugsatfda_docs/...,pre96,Failure
123,20182,http://www.accessdata.fda.gov/drugsatfda_docs/...,pre96,Success
124,20177,http://www.accessdata.fda.gov/drugsatfda_docs/...,pre96,NaN
...,...,...,...,...
7655,22520,http://www.accessdata.fda.gov/drugsatfda_docs/...,2001,NaN
7656,43039,http://www.accessdata.fda.gov/drugsatfda_docs/...,2001,NaN
7657,43041,http://www.accessdata.fda.gov/drugsatfda_docs/...,2002,NaN
7658,22536,http://www.accessdata.fda.gov/drugsatfda_docs/...,2003,NaN


In [132]:
for idx,row in url_data[:5].iterrows():
    print(row.loc[idx])

KeyError: 0

In [83]:
data['results'][0]['submissions'][2]['application_docs']

[{'id': '8876',
  'url': 'http://www.accessdata.fda.gov/drugsatfda_docs/appletter/2002/76200.apd.pdf',
  'date': '20020319',
  'type': 'Letter'},
 {'id': '22545',
  'url': 'http://www.accessdata.fda.gov/drugsatfda_docs/nda/2002/076200_s000_AcetaminophenTOC.cfm',
  'date': '20060420',
  'type': 'Review'}]

In [84]:
data['results'][0]['submissions'][2]['application_docs'][0]

{'id': '8876',
 'url': 'http://www.accessdata.fda.gov/drugsatfda_docs/appletter/2002/76200.apd.pdf',
 'date': '20020319',
 'type': 'Letter'}

In [176]:
url_data.year.unique()

array(['2002', '2004', '2013', '2015', '2005', '2003', '2006', '2009',
       '99', 'pre96', '2007', '2010', '2021', '2001', '97', '2000',
       '2024', '2011', '96', 'TherapeuticBiologicApplications', '2016',
       '2008', '2019', '2018', '2023', '2025', '2014', '2012', '1997',
       '98', '2020', '2022', '2017', '1999', '1998', 'products', 'viagra',
       'rems', 'daf', '1996'], dtype=object)

In [171]:
test = "https://www.accessdata.fda.gov/drugsatfda_docs/ANDA/98/viagra/viagra_toc.cfm"

In [172]:
test.lower().split("nda")[1].split("/")[1]

'98'

In [10]:
from langchain_community.document_loaders import PyMuPDFLoader
import pymupdf



In [21]:

file_path = "/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files/2017_52670_Ascor (ascorbic acid) Injection/clinical_pharmacology_biopharmaceutics_review(s).pdf"
# loader = PyMuPDFLoader(file_path)

In [3]:
docs = loader.load()
docs[0]


Document(metadata={'producer': 'Adobe Acrobat Pro 11.0.0 Paper Capture Plug-in', 'creator': '', 'creationdate': '2018-01-01T20:59:12-05:00', 'source': "/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files/2017_52670_Ascor (ascorbic acid) Injection/clinical_pharmacology_biopharmaceutics_review(s).pdf", 'file_path': "/Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files/2017_52670_Ascor (ascorbic acid) Injection/clinical_pharmacology_biopharmaceutics_review(s).pdf", 'total_pages': 50, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2018-01-01T20:59:57-05:00', 'trapped': '', 'modDate': "D:20180101205957-05'00'", 'creationDate': "D:20180101205912-05'00'", 'page': 0}, page_content='CENTER FOR DRUG EVALUATION AND \nRESEARCH \n \n \nAPPLICATION NUMBER: \n \n209112Orig1s000 \n \n \nCLINICAL PHARMACOLOGY AND \nBIOPHARMACEUTICS REVIEW(S)')

In [11]:
text = pymupdf.open(file_path)

In [8]:
full_text = ""
for page_num in range(docs.page_count):
    page = doc.load_page(page_num) # or doc[page_num]
    text = page.get_text("text") # "text" extracts plain text with line breaks
    full_text += text# docs[2].

'2\n7\nDRUG INTERACTIONS\n7.1 Antibiotics\nAscorbic acid may decrease activities of erythromycin, kanamycin, streptomycin, doxycycline, and lincomycin. \nBleomycin is inactivated in vitro by ascorbic acid.  If the antibiotic efficacy is suspected to be decreased by \nconcomitant administration of ASCOR, discontinue ASCOR administration.\n7.2 Amphetamine & Other Drugs Affected by Urine Acidification\nAscorbic acid may acidify the urine and lower serum concentrations of amphetamine by increasing renal \nexcretion (as reflected by changes in amphetamine urine recovery rates).  In case of decreased amphetamine \nefficacy discontinue ASCOR administration.  Standard monitoring of therapy is warranted.\nIn addition, acidification of urine by ascorbic acid will alter the excretion of certain drugs affected by the pH of \nthe urine (e.g. fluphenazine, \n when administered concurrently. It has been reported that concurrent \nadministration of ascorbic acid and fluphenazine has resulted in decrea

In [22]:
doc = pymupdf.open(file_path)

In [23]:
doc[2]

page 2 of /Users/shubhamagrawal/Documents/MS fall 25/MS fall'25/ra/data/final_pdf_files/2017_52670_Ascor (ascorbic acid) Injection/clinical_pharmacology_biopharmaceutics_review(s).pdf

In [17]:
text.get_text()

AttributeError: 'Document' object has no attribute 'get_text'

In [28]:
main_df = pd.read_csv('/Users/shubhamagrawal/Documents/MS fall 25/MS fall\'25/ra/data/main_df.csv')

In [29]:
main_df.head()

,json_id,year,url,status,pdf_count,path_index,drug_type,pdf_content_length,pymupdf_parsaed
0,48910,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,0,NaN,NaN,0,0
1,57420,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_57420_others_103628Orig1s000rev,others,74010,1
2,20189,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_20189_others_018998_S050_VASOTEC_AP,others,0,0
3,40848,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,1,96_40848_others_018998_S034_VASOTEC_AP,others,0,0
4,48910,96,http://www.accessdata.fda.gov/drugsatfda_docs/...,Success,0,NaN,NaN,0,0
